
# **Optimización de Portafolios y Frontera Eficiente**

Este cuaderno explora la construcción de la **frontera eficiente** dentro del marco de **Media-Varianza**, utilizando la clase `MeanRisk` de la biblioteca `skfolio`.

La frontera eficiente, basada en los principios de **Harry Markowitz (1952)**, representa el conjunto de portafolios óptimos que maximizan el retorno esperado para un nivel dado de riesgo o, alternativamente, minimizan el riesgo para un nivel determinado de retorno esperado.

Para ello, identificaremos un **conjunto de carteras óptimas** en la **frontera de Pareto**, evaluando su desempeño y comparándolas en términos de retorno y riesgo.


Antes de continuar, asegurémonos de que `skfolio` está instalado en el entorno de Python. Si no lo tienes instalado, ejecuta el siguiente comando:

In [2]:
!pip install skfolio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 10.6 MB/s eta 0:00:00


## **Carga y preparación de datos**
En esta sección, importamos las bibliotecas necesarias y cargamos un conjunto de datos compuesto por los precios diarios de 20 activos del índice S&P 500. Los datos abarcan el período del 2 de enero de 1990 al 28 de diciembre de 2022.

Para garantizar la validez de la estrategia y evitar la **fuga de datos**, se divide el conjunto de datos en subconjuntos de entrenamiento y prueba sin realizar mezclas aleatorias.



In [3]:
# Importación de bibliotecas necesarias
import numpy as np
from plotly.io import show  # Para la visualización de gráficos interactivos
from sklearn.model_selection import train_test_split  # Para dividir los datos en entrenamiento y prueba

# Importación de Skfolio para análisis de portafolios y optimización
from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset  # Conjunto de datos S&P 500
from skfolio.optimization import MeanRisk  # Clase de optimización de Media-Varianza
from skfolio.preprocessing import prices_to_returns  # Conversión de precios a rendimientos

# Carga del conjunto de datos de precios históricos del S&P 500
prices = load_sp500_dataset()

# Transformación de precios en rendimientos para el análisis
X = prices_to_returns(prices)

# División de los datos en conjunto de entrenamiento (67%) y prueba (33%), sin mezclar
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

## **Construcción del Modelo de Media-Varianza**
En esta sección, implementamos un modelo de **Media-Varianza**, utilizando la clase `MeanRisk` de `skfolio`.

Este modelo optimiza carteras en función del **riesgo (varianza)** y busca construir una **frontera eficiente** con un número determinado de portafolios óptimos.

Parámetros Clave:
- `risk_measure=RiskMeasure.VARIANCE`: Define la varianza como la métrica de riesgo a minimizar.
- `efficient_frontier_size=30`: Se generan **30 carteras óptimas** distribuidas a lo largo de la frontera eficiente.
- `portfolio_params=dict(name="Variance")`: Etiqueta la cartera optimizada como **"Variance"** para facilitar su identificación en análisis posteriores.

In [4]:
# Creación del modelo de optimización Media-Varianza
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    efficient_frontier_size=30,
    portfolio_params=dict(name="Variance"),
)

# Ajuste del modelo utilizando los datos de entrenamiento (rendimientos históricos)
model.fit(X_train)

# Verificación de la forma de la matriz de pesos obtenida (Número de carteras x Número de activos)
print(model.weights_.shape)

(30, 20)


## **Predicción del Modelo en los Conjuntos de Entrenamiento y Prueba**
En esta etapa, utilizamos el método `predict` para obtener la composición de las **30 carteras** optimizadas dentro de la frontera eficiente.

El resultado devuelto por `predict` es una instancia de la clase `Population`, que agrupa múltiples carteras (`Portfolio`) optimizadas en la frontera eficiente.



In [5]:
# Predicción de las carteras óptimas en el conjunto de entrenamiento
population_train = model.predict(X_train)  # Devuelve una población de 30 carteras basadas en X_train

# Predicción de las carteras óptimas en el conjunto de prueba (evaluación fuera de muestra)
population_test = model.predict(X_test)  # Devuelve una población de 30 carteras basadas en X_test

## **Análisis de la Frontera Eficiente y Evaluación de Carteras**
Para visualizar y comparar el desempeño de las carteras en el conjunto de **entrenamiento** y **prueba**, se realiza lo siguiente:



In [6]:
# Etiquetado de las carteras según el conjunto de datos utilizado
population_train.set_portfolio_params(tag="Train")  # Marca las carteras del conjunto de entrenamiento
population_test.set_portfolio_params(tag="Test")  # Marca las carteras del conjunto de prueba

# Concatenación de ambas poblaciones en un solo objeto para su análisis comparativo
population = population_train + population_test

# Generación del gráfico de análisis de medidas de desempeño
fig = population.plot_measures(
    x=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION,  # Riesgo: Desviación estándar anualizada (Eje X)
    y=PerfMeasure.ANNUALIZED_MEAN,  # Retorno esperado anualizado (Eje Y)
    color_scale=RatioMeasure.ANNUALIZED_SHARPE_RATIO,  # Eficiencia riesgo-retorno basada en Sharpe Ratio (Color)
    hover_measures=[  # Información adicional al pasar el cursor sobre un punto en el gráfico
        RiskMeasure.MAX_DRAWDOWN,  # Máxima caída desde el pico
        RatioMeasure.ANNUALIZED_SORTINO_RATIO,  # Sortino Ratio anualizado (retorno ajustado al riesgo de pérdidas)
    ],
)

# Mostrar la visualización interactiva del gráfico
show(fig)

Veamos la composición de las 30 carteras:



In [7]:
population_train.plot_composition()

Imprimamos el Sharpe Ratio de las 30 carteras del conjunto de prueba:



In [8]:
population_test.measures(measure=RatioMeasure.ANNUALIZED_SHARPE_RATIO)

array([0.91785162, 0.93000375, 0.9401232 , 0.95027284, 0.96861483,
       0.98463685, 0.9983657 , 1.00992555, 1.01943433, 1.02702131,
       1.032813  , 1.03704654, 1.03993049, 1.04204356, 1.0430764 ,
       1.04289017, 1.04158213, 1.03774863, 1.02943678, 1.02092595,
       1.01241175, 1.00356473, 0.97964062, 0.93749217, 0.87986871,
       0.78090544, 0.68550154, 0.59858003, 0.53398775, 0.55455742])

Por último, podemos mostrar un resumen completo de las 30 carteras evaluadas en el conjunto de pruebas:



In [9]:
population.summary()

,ptf0 - Variance,ptf1 - Variance,ptf2 - Variance,ptf3 - Variance,ptf4 - Variance,ptf5 - Variance,ptf6 - Variance,ptf7 - Variance,ptf8 - Variance,ptf9 - Variance,...,ptf20 - Variance,ptf21 - Variance,ptf22 - Variance,ptf23 - Variance,ptf24 - Variance,ptf25 - Variance,ptf26 - Variance,ptf27 - Variance,ptf28 - Variance,ptf29 - Variance
Mean,0.062%,0.065%,0.068%,0.071%,0.074%,0.076%,0.079%,0.082%,0.085%,0.088%,...,0.086%,0.088%,0.088%,0.087%,0.086%,0.083%,0.081%,0.079%,0.078%,0.090%
Annualized Mean,15.66%,16.38%,17.10%,17.82%,18.54%,19.26%,19.99%,20.71%,21.43%,22.15%,...,21.71%,22.10%,22.16%,21.92%,21.59%,20.98%,20.37%,19.78%,19.56%,22.73%
Variance,0.011%,0.011%,0.011%,0.011%,0.012%,0.012%,0.012%,0.013%,0.014%,0.014%,...,0.018%,0.019%,0.020%,0.022%,0.024%,0.029%,0.035%,0.043%,0.053%,0.067%
Annualized Variance,2.76%,2.78%,2.81%,2.86%,2.94%,3.03%,3.15%,3.28%,3.44%,3.62%,...,4.60%,4.85%,5.12%,5.47%,6.02%,7.22%,8.83%,10.92%,13.41%,16.80%
Semi-Variance,0.0054%,0.0055%,0.0055%,0.0056%,0.0058%,0.0060%,0.0062%,0.0065%,0.0068%,0.0072%,...,0.0091%,0.0096%,0.010%,0.011%,0.012%,0.014%,0.017%,0.021%,0.026%,0.034%
Annualized Semi-Variance,1.37%,1.37%,1.39%,1.42%,1.46%,1.50%,1.56%,1.63%,1.71%,1.80%,...,2.28%,2.41%,2.54%,2.70%,2.96%,3.52%,4.29%,5.30%,6.55%,8.50%
Standard Deviation,1.05%,1.05%,1.06%,1.07%,1.08%,1.10%,1.12%,1.14%,1.17%,1.20%,...,1.35%,1.39%,1.43%,1.47%,1.55%,1.69%,1.87%,2.08%,2.31%,2.58%
Annualized Standard Deviation,16.62%,16.67%,16.76%,16.92%,17.13%,17.41%,17.74%,18.12%,18.55%,19.03%,...,21.45%,22.02%,22.63%,23.38%,24.53%,26.87%,29.72%,33.05%,36.63%,40.99%
Semi-Deviation,0.74%,0.74%,0.74%,0.75%,0.76%,0.77%,0.79%,0.81%,0.82%,0.85%,...,0.95%,0.98%,1.00%,1.03%,1.08%,1.18%,1.30%,1.45%,1.61%,1.84%
Annualized Semi-Deviation,11.69%,11.72%,11.79%,11.91%,12.07%,12.27%,12.51%,12.78%,13.09%,13.43%,...,15.11%,15.52%,15.92%,16.42%,17.19%,18.77%,20.71%,23.02%,25.59%,29.16%


## **Optimización de Carteras con Restricción de Rentabilidad Mínima**
En esta sección, exploramos una alternativa a la construcción de la frontera eficiente. En lugar de definir un número fijo de carteras con `efficient_frontier_size`, podemos imponer restricciones explícitas sobre el retorno mínimo esperado de cada cartera.

**Objetivo**: Encontrar **cinco carteras óptimas** que minimicen la varianza bajo las siguientes restricciones de retorno mínimo anualizado:
- 15%
- 20%
- 25%
- 30%
- 35%

Debido a que los datos están en frecuencia diaria, convertimos estas tasas de retorno anualizadas a retornos diarios dividiéndolas por **252 días de mercado por año**.



In [10]:
# Definimos un modelo de optimización basado en Media-Varianza con restricción de retorno mínimo
model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,  # Minimizamos la varianza (riesgo)
    min_return=np.array([0.15, 0.20, 0.25, 0.30, 0.35]) / 252,  # Restricciones de rentabilidad mínima en términos diarios
    portfolio_params=dict(name="Variance"),  # Etiqueta identificativa del modelo
)

# Ajustamos el modelo al conjunto de entrenamiento y predecimos las carteras óptimas
population = model.fit_predict(X_train)

# Graficamos las carteras en el espacio riesgo-retorno para analizar su desempeño
population.plot_measures(
    x=RiskMeasure.ANNUALIZED_STANDARD_DEVIATION,  # Riesgo: Desviación estándar anualizada (Eje X)
    y=PerfMeasure.ANNUALIZED_MEAN,  # Retorno esperado anualizado (Eje Y)
    color_scale=RatioMeasure.ANNUALIZED_SHARPE_RATIO,  # Color basado en el Sharpe Ratio anualizado
    hover_measures=[  # Información adicional al pasar el cursor
        RiskMeasure.MAX_DRAWDOWN,  # Máxima caída desde el pico
        RatioMeasure.ANNUALIZED_SORTINO_RATIO,  # Sortino Ratio anualizado (retorno ajustado al riesgo de pérdidas)
    ],
)

## **Conclusión**
En este cuaderno, exploramos la construcción de la **frontera eficiente** utilizando la metodología **Media-Varianza**, aplicando diferentes enfoques para optimizar carteras en función del riesgo y el retorno esperado.

**Optimización de la Frontera Eficiente**:

- Utilizamos la clase `MeanRisk` para calcular un conjunto de 30 carteras que pertenecen a la frontera eficiente.
- Analizamos la relación entre la **desviación estándar anualizada (riesgo)** y la rentabilidad esperada anualizada, utilizando **Sharpe Ratio** como métrica de referencia.

**Optimización con Restricción de Rentabilidad Mínima**:

- En lugar de definir un número fijo de carteras, impusimos restricciones de **rentabilidad mínima esperada** en un rango del 15% al 35% anualizado.
- Este enfoque permite encontrar carteras óptimas que cumplan con objetivos específicos de retorno, minimizando la varianza dentro de los límites establecidos.

**Análisis Comparativo**:

- Visualizamos la **distribución de las carteras** en función de su riesgo y retorno, lo que permitió identificar aquellas con mejor desempeño en términos del Sharpe Ratio y **Sortino Ratio**.
- Al contrastar ambas metodologías, observamos que imponer una restricción de rentabilidad mínima puede generar carteras con mayor riesgo, pero con potenciales retornos más atractivos.